# Floreano Experiment in the NRP

## Initialize the Virtual Coach with Storage Server credentials

In [1]:
# disable global logging from the virtual coach
import logging
logging.disable(logging.INFO)
logging.getLogger('rospy').propagate = False
logging.getLogger('rosout').propagate = False

In [2]:
import os
import sys
import time
import pandas
import shutil
import plot_utils
import evolution_utils

import numpy as np
import matplotlib.pyplot as plt

from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach

%matplotlib inline
vc = VirtualCoach(environment='local', storage_username='nrpuser', storage_password='password')

## Helper Functions
Some helper functions to calculate the fitness function, plot the robot's trajectory and the wheel speeds. These functions are specific to this experiment.

In [3]:
display_episode_tf = """@nrp.Robot2Neuron()
def display_episode_number(t):
    clientLogger.advertise('%s')
"""

## Run Experiment

Run the evolutionary experiment after specifying the number of generations and individuals per generation

In [4]:
class FloreanoExperiment(object):
    
    def __init__(self, experiment_name, population, generations):
        self.experiment_name = experiment_name
        self.experiment_dir = os.environ['HOME'] + '/.opt/nrpStorage/' + self.experiment_name
        self.last_status = [None]
        self.population = population
        self.fitness_log = []
        self.sim = None
        self.started = False
        self.generations = generations
        self.sim_data = []

        # create directories for each generation to store results in 
        for i in range(generations):
            os.mkdir(self.experiment_dir + '/generation_{}'.format(i))

    def wait_condition(self, timeout, condition):
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(0.25)
            if condition(self.last_status[0]):
                return
        raise Exception('Condition check failed')

    def on_status(self, status):
        self.last_status[0] = status
        
    def save_simulation_data(self, generation, trial):
        """
        Saves the simulation csv data to the respective individual's directory inside the
        experiment's directory
        """
        self.sim.save_csv()
        csv_dir = [s for s in os.listdir(self.experiment_dir) if "csv_records" in s][0]
        individual_dir = self.experiment_dir + '/generation_{}'.format(generation) + '/individual_{}'.format(trial)
        shutil.move(self.experiment_dir + '/' + csv_dir, individual_dir)
        np.save(individual_dir + '/genetic_string', self.population[trial])

        wheel_speeds = evolution_utils.get_wheel_speeds(individual_dir)
        np.save(individual_dir + '/wheel_speeds', wheel_speeds)
        trajectory = evolution_utils.get_trajectory(individual_dir)
        collision = evolution_utils.correct_for_collisions(individual_dir)
        # plot trajectory, spikes and wheel speeds
        plot_utils.plot_spikes(individual_dir)
        plot_utils.plot_trajectory(individual_dir, trajectory)
        plot_utils.plot_wheel_speeds(individual_dir, wheel_speeds)

        # save fitness value
        if collision:
            fitness_value = -1
        else:
            fitness_value = evolution_utils.fitness_function(wheel_speeds)
        np.save(individual_dir + '/fitness_value', fitness_value)


    def run_experiment(self):
        try:
            self.sim = vc.launch_experiment('floreano_0')
        except:
            time.sleep(1)
        self.sim.register_status_callback(self.on_status)
        for i in range(self.generations):
            for j in range(60):
                print "Generation {}, Individual {}".format(i, j),
                sys.stdout.flush()
                genetic_string = ','.join(str(x) for x in population[j].ravel())
                self.sim.edit_brain(evolution_utils.brain % genetic_string)
                self.sim.add_transfer_function(display_episode_tf % "Generation {}, Individual {}".format(i, j))
                self.sim.start()
                # run simulation for 40 seconds
                self.wait_condition(10000, lambda x: x['simulationTime'] > 40)
                self.sim.pause()
                self.save_simulation_data(i, j)
                self.sim.reset('full')
                self.wait_condition(1000, lambda x: x['state'] == 'paused' and x['simulationTime'] == 0)
            evolution_utils.get_top_performers(self.experiment_dir + '/generation_{}'.format(i))
            self.population = evolution_utils.evolve_new_generation(self.experiment_dir + '/generation_{}'.format(i))
            file = open("data_dump.txt","w")
            file.write(str(floreano_experiment.sim_data))
            file.write("\n")
            file.close()

In [ ]:
population = np.random.randint(2, size=(60, 10, 29)) # random population of 10 binary genetic strings
floreano_experiment = FloreanoExperiment('floreano_0', population, 30)
floreano_experiment.run_experiment()

[{u'gzweb': {u'assets': u'http://localhost:8080/assets',
             u'nrp-services': u'http://localhost:8080',
             u'videoStreaming': u'http://localhost:8080/webstream/',
             u'websocket': u'ws://localhost:8080/gzbridge'},
  u'id': u'localhost',
  u'rosbridge': {u'websocket': u'ws://localhost:8080/rosbridge'},
  u'serverJobLocation': u'local'}]
Generation 0, Individual 0 400
Generation 0, Individual 1 Generation 0, Individual 2 Generation 0, Individual 3 400
Generation 0, Individual 4 400
Generation 0, Individual 5 400
Generation 0, Individual 6 400
Generation 0, Individual 7 400
Generation 0, Individual 8 400
Generation 0, Individual 9 400
Generation 0, Individual 10 Generation 0, Individual 11 400
Generation 0, Individual 12 400
Generation 0, Individual 13 400
Generation 0, Individual 14 400
Generation 0, Individual 15 400
Generation 0, Individual 16 400
Generation 0, Individual 17 400
Generation 0, Individual 18 400
Generation 0, Individual 19 400
Generation 0, I

## Analyze Experiment Results

In [ ]:
fig, axes = plt.subplots(len(floreano_experiment.sim_data), 2)
for i in range(len(floreano_experiment.sim_data)):
    axes[i, 0].set_ylim(-3, 3)
    axes[i, 0].set_xlim(-3.9, 3.9)
    axes[i, 0].set_xticks([], [])
    axes[i, 0].set_yticks([], [])
    x_axis = [x[0] for x in floreano_experiment.sim_data[i]['trajectory'][2:]]
    y_axis = [y[1] for y in floreano_experiment.sim_data[i]['trajectory'][2:]]
    axes[i, 0].plot([float(x) for x in x_axis], [float(y) for y in y_axis])

    left_wheel = ([float(t[1]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    right_wheel = ([float(t[2]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    axes[i, 1].plot(range(len(left_wheel)), left_wheel, 'b', label='Left Wheel')
    axes[i, 1].plot(range(len(right_wheel)), right_wheel, 'r', label='Right Wheel')
    axes[i, 1].set_ylim(-5, 5)
    axes[i, 1].set_xlabel('Time [ms]')
    axes[i, 1].set_ylabel('Speed m/s')

axes[0, 0].set_title('Robot Trajectory')
axes[0, 1].set_title('Wheel Speeds')
fig.set_figheight(25)
fig.set_figwidth(10)